In [1]:
import numpy as np
import src.solutions.eigenmodes_part1 as eigen_part1
import src.solutions.direct_diffusion as direct_diffusion
import src.solutions.leapfrog as leapfrog
import src.visualizations as vis


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.cm as cm
import matplotlib.colors as mcolors

In [3]:
N = 5

In [4]:
# initial_matrix = eigen_part1.create_init_matrix_a(N)
# print(initial_matrix)

### Steady State Diffusion with a Direct solver
Uses a dependency matrix incorporating all the dependencies and a b that enforces the boundary conditions and source.  
This is implemented on a circular grid where the edges of the circle are fixed at 0. The grid is solved with different discretization steps. 

In [ ]:
# N = 200
nntjes = [200, 40, 20]
source_location = (0.6, 1.2)
diameter = 4

# create the grid and plot the converged grid 
converged_grids = direct_diffusion.direct_diffusion(nntjes, source_location, diameter)
vis.plot_diffusion_circle(converged_grids, nntjes)

## Leapfrog Method on Spring dynamics
using the leapfrog method to discretize spring dynamics, simulate for different spring constants (k)

#### Parameter Values

In [6]:
#parameters first experiment
ks = [16, 8, 4, 2, 1]
m=1
x = 1
v_0 = 0
deltat = 0.01

# extra parameters second experiment
freqs =  [1.4, 1.2, 1, 0.8]
k=1
time = 18
xs = np.linspace(-2, 2, 9, endpoint=True)

#### Simulation pure harmonic oscillator, visualizing position-velocity plot

In [ ]:
data_per_k = leapfrog.harmonic_oscillator_leapfrog(ks, deltat, x, v_0, m)

vis.vis_harmonic_oscillator(data_per_k)

#### Simulation Spring dynamics with extra time-dependent sinusoidal force, Phase plot with position vs velocity

In [ ]:
# saves all simulation steps for every frequency of extra force 
phases_for_freqs = dict()

# iterate over all frequencies for the time-dependent extra force
for freq in freqs:
    data_per_x0 = leapfrog.harmonic_oscillator_extra_force(k, deltat, xs, v_0, m, freq, time)
    phases_for_freqs[freq] = data_per_x0

# makes a phase plot for every frequency
vis.vis_phase_oscillator(phases_for_freqs, freqs)
